In [1]:
import bw2data, bw2calc, bw2io
import yaml

11:26:39+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [65]:
if "commentary_response" in bw2data.projects:
    bw2data.projects.delete_project("commentary_response", delete_dir=True)

bw2data.projects.set_current("commentary_response")

fp = "technosphere.yaml"
with open(fp, "r") as stream:
    technosphere = yaml.load(stream, Loader=yaml.FullLoader)

fp = "technosphere_with_EoL.yaml"
with open(fp, "r") as stream:
    technosphere_w_EoL = yaml.load(stream, Loader=yaml.FullLoader)

fp = "biosphere.yaml"
with open(fp, "r") as stream:
    biosphere = yaml.load(stream, Loader=yaml.FullLoader)

fp = "static_lcia_method.yaml"
with open(fp, "r") as stream:
    lcia = yaml.load(stream, Loader=yaml.FullLoader)

In [66]:
bio = bw2data.Database("biosphere")
bio.write(biosphere)

100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 25890.77it/s]

12:56:25+0100 [info     ] Vacuuming database            


In [67]:
tech = bw2data.Database("lithium_supply_chain")
tech.write(technosphere)

100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 26658.71it/s]

12:56:26+0100 [info     ] Vacuuming database            


In [68]:
tech = bw2data.Database("lithium_supply_chain_w_EoL")
tech.write(technosphere_w_EoL)

100%|██████████████████████████████████████████| 7/7 [00:00<00:00, 24405.76it/s]

12:56:26+0100 [info     ] Vacuuming database            


In [6]:
method_key = ("Water scarcity",)

if method_key in bw2data.methods:
    del bw2data.methods[method_key]
    
lcia_data = [(bw2data.get_activity(f).id, cf) for f, cf in lcia]

method = bw2data.Method(method_key)
metadata = {
    "unit": "kg deprived water-eq.",
}
method.register(**metadata)
method.write(lcia_data)

In [7]:
bw2data.Method(method_key).load()

[(251654880983322624, 39.5),
 (251654880983322625, 39.5),
 (251654880983322626, 39.5)]

In [8]:
fu = bw2data.get_activity(("lithium_supply_chain", "cathode material production"))

In [9]:
fu

'CAM production' (kg, DE, None)

In [10]:
lca = bw2calc.LCA(
    demand={fu: 1},
    method=method_key
)

In [11]:
lca.lci()

In [12]:
lca.lcia()

In [13]:
lca.score

77285.70085534752

In [14]:
import numpy as np
import scipy.sparse as sp
from IPython.display import Latex


def sparse_to_latex(matrix, fmt="{:g}", row_labels=None, col_labels=None):
    """
    Convert a SciPy sparse matrix (or numpy array) to a LaTeX array environment,
    with optional row and column labels that preserve spaces using \\text{}.
    Zero values are displayed as "-".
    """

    # Convert to dense numpy array
    if sp.issparse(matrix):
        A = matrix.toarray()
    else:
        A = np.asarray(matrix)

    # Ensure A is 2D
    if A.ndim == 0:
        A = A.reshape(1, 1)
    elif A.ndim == 1:
        A = A.reshape(1, -1)

    n_rows, n_cols = A.shape

    # Validate labels
    if row_labels is not None and len(row_labels) != n_rows:
        raise ValueError("row_labels length does not match number of rows")
    if col_labels is not None and len(col_labels) != n_cols:
        raise ValueError("col_labels length does not match number of columns")

    # Wrap labels so LaTeX preserves spaces
    if row_labels is not None:
        row_labels = [f"\\text{{{lbl}}}" for lbl in row_labels]
    if col_labels is not None:
        col_labels = [f"\\text{{{lbl}}}" for lbl in col_labels]

    rows = []

    # Column header row
    if col_labels is not None:
        if row_labels is not None:
            header = " & " + " & ".join(col_labels)
        else:
            header = " & ".join(col_labels)
        rows.append(header)

    # Data rows
    for i, row in enumerate(A):
        formatted_row = " & ".join(
            "-" if x == 0 else fmt.format(x)   # HERE is the new logic
            for x in row
        )

        if row_labels is not None:
            rows.append(row_labels[i] + " & " + formatted_row)
        else:
            rows.append(formatted_row)

    # Alignment: extra column only if row labels exist
    if row_labels is not None:
        n_cols_total = n_cols + 1
    else:
        n_cols_total = n_cols

    align = "c" * n_cols_total

    body = " \\\\\n".join(rows)
    latex = (
        "\\[\n"
        "\\begin{array}{" + align + "}\n"
        + body +
        "\n\\end{array}\n\\]"
    )

    return latex



In [15]:
Latex(
    sparse_to_latex(
        lca.demand_array,
        #row_labels=[bw2data.get_activity(x)["name"] for x in lca.activity_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['reference product']} [{bw2data.get_activity(x)['unit']}]" for x in lca.product_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [16]:
Latex(
    sparse_to_latex(
        lca.technosphere_matrix,
        row_labels=[f"{bw2data.get_activity(x)['reference product']} ({bw2data.get_activity(x)['location']})" for x in lca.product_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [17]:
Latex(
    sparse_to_latex(
        lca.biosphere_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.biosphere_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [18]:
Latex(
    sparse_to_latex(
        lca.inventory,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.biosphere_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [19]:
Latex(
    sparse_to_latex(
        lca.characterization_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.biosphere_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.biosphere_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [19]:
Latex(
    sparse_to_latex(
        lca.characterized_inventory,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.biosphere_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [20]:
from edges import EdgeLCIA

In [21]:
rmethod = {
    "name": "Water scarcity",
    "version": "1.0",
    "description": "Example LCIA method for water scarcity",
    "unit": "kg deprived-eq.",
    "strategies": [
        "map_exchanges"
    ],
    "exchanges": [
        {
            "supplier": {
                "name": "water",
                "operator": "contains",
                "matrix": "biosphere"
            },
            "consumer": {
                "location": "CL",
                "matrix": "technosphere"
            },
            "value": 45.5
        },
        {
            "supplier": {
                "name": "water",
                "operator": "contains",
                "matrix": "biosphere"
            },
            "consumer": {
                "location": "CN",
                "matrix": "technosphere"
            },
            "value": 6.3
        },
        {
            "supplier": {
                "name": "water",
                "operator": "contains",
                "matrix": "biosphere"
            },
            "consumer": {
                "location": "DE",
                "matrix": "technosphere"
            },
            "value": 2.1  # CF value
        }
    ]
}


In [22]:
rlca = EdgeLCIA(
    demand={fu: 1},
    method=rmethod
)

No handlers could be found for logger "edges.edgelcia.EdgeLCIA"


In [23]:
rlca.lci()
rlca.apply_strategies()
rlca.evaluate_cfs()
rlca.lcia()
rlca.score

Mapping exchanges: 100%|███████████████████████| 3/3 [00:00<00:00, 21006.53it/s]


51047.50062954128

In [24]:
Latex(
    sparse_to_latex(
        rlca.lca.inventory,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.biosphere_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [25]:
Latex(
    sparse_to_latex(
        rlca.characterization_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.biosphere_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['location']}]" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [26]:
Latex(
    sparse_to_latex(
        rlca.characterized_inventory,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in lca.biosphere_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [50]:
tech_method = {
    "name": "Electricity tracking",
    "version": "1.0",
    "description": "Example LCIA method for electricity tracking",
    "unit": "kWh-eq.",
    "strategies": [
        "map_exchanges"
    ],
    "exchanges": [
        {
            "supplier": {
                "unit": "kWh",
                "matrix": "technosphere"
            },
            "consumer": {
                "location": "CL",
                "matrix": "technosphere"
            },
            "value": 10
        },
        {
            "supplier": {
                "unit": "kWh",
                "matrix": "technosphere"
            },
            "consumer": {
                "location": "CN",
                "matrix": "technosphere"
            },
            "value": 5
        },
        {
            "supplier": {
                "unit": "kWh",
                "location": "DE",
                "matrix": "technosphere"
            },
            "consumer": {
                "location": "DE",
                "matrix": "technosphere"
            },
            "value": 1
        },
        {
            "supplier": {
                "unit": "kWh",
                "location": "CL",
                "matrix": "technosphere"
            },
            "consumer": {
                "location": "DE",
                "matrix": "technosphere"
            },
            "value": 10
        }
    ]
}


In [51]:
tech_lca = EdgeLCIA(
    demand={fu: 1},
    method=tech_method
)

In [52]:
tech_lca.lci()
tech_lca.apply_strategies()
tech_lca.evaluate_cfs()
tech_lca.lcia()
tech_lca.score

Mapping exchanges: 100%|███████████████████████| 4/4 [00:00<00:00, 25614.07it/s]


279.3000000759959

In [53]:
Latex(
    sparse_to_latex(
        tech_lca.technosphere_flow_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['location']}]" for x in lca.activity_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['location']}]" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [54]:
Latex(
    sparse_to_latex(
        tech_lca.characterization_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [56]:
Latex(
    sparse_to_latex(
        tech_lca.characterized_inventory,
        row_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [69]:
fu_w_EoL = bw2data.get_activity(("lithium_supply_chain_w_EoL", "cathode material production"))

In [70]:
tech_lca_w_EoL = EdgeLCIA(
    demand={fu_w_EoL: 1},
    method=tech_method
)

In [71]:
tech_lca_w_EoL.lci()
tech_lca_w_EoL.apply_strategies()
tech_lca_w_EoL.evaluate_cfs()
tech_lca_w_EoL.lcia()
tech_lca_w_EoL.score

Mapping exchanges: 100%|███████████████████████| 4/4 [00:00<00:00, 20841.26it/s]


258.454545173517

In [72]:
Latex(
    sparse_to_latex(
        tech_lca_w_EoL.lca.technosphere_matrix,
        row_labels=[f"{bw2data.get_activity(x)['reference product']} ({bw2data.get_activity(x)['location']})" for x in tech_lca_w_EoL.lca.product_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['unit']}]" for x in tech_lca_w_EoL.lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [74]:
Latex(
    sparse_to_latex(
        tech_lca_w_EoL.technosphere_flow_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['location']}]" for x in tech_lca_w_EoL.lca.activity_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['location']}]" for x in tech_lca_w_EoL.lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [75]:
Latex(
    sparse_to_latex(
        tech_lca_w_EoL.characterization_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in tech_lca_w_EoL.lca.activity_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in tech_lca_w_EoL.lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [76]:
Latex(
    sparse_to_latex(
        tech_lca_w_EoL.characterized_inventory,
        row_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in tech_lca_w_EoL.lca.activity_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in tech_lca_w_EoL.lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [77]:
import numpy as np
from scipy.sparse import csr_matrix

def normalize_negative_reference_activities(technosphere_matrix: csr_matrix) -> csr_matrix:
    """
    Find activities whose reference flow (diagonal) is negative and
    flip the sign of the entire row so that:
      - the diagonal becomes positive
      - all their outgoing exchanges change sign.

    This should be called *before* building the technosphere flow matrix.
    """
    A = technosphere_matrix.tocsr(copy=True)

    diag = A.diagonal()
    neg_ref_idx = np.where(diag < 0)[0]

    # Nothing to do
    if neg_ref_idx.size == 0:
        return A

    for i in neg_ref_idx:
        start, end = A.indptr[i], A.indptr[i + 1]
        A.data[start:end] *= -1

    return A

In [78]:
tech_lca_w_EoL = EdgeLCIA(
    demand={fu_w_EoL: 1},
    method=tech_method
)

In [81]:
tech_lca_w_EoL.lca.technosphere_matrix = normalize_negative_reference_activities(tech_lca_w_EoL.lca.technosphere_matrix)

In [82]:
tech_lca_w_EoL.lci()

In [83]:
tech_lca_w_EoL.apply_strategies()
tech_lca_w_EoL.evaluate_cfs()
tech_lca_w_EoL.lcia()
tech_lca_w_EoL.score

Mapping exchanges: 100%|███████████████████████| 4/4 [00:00<00:00, 21732.15it/s]


258.454545173517

In [85]:
Latex(
    sparse_to_latex(
        tech_lca_w_EoL.technosphere_flow_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['location']}]" for x in tech_lca_w_EoL.lca.activity_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['name']} [{bw2data.get_activity(x)['location']}]" for x in tech_lca_w_EoL.lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [52]:
Latex(
    sparse_to_latex(
        tech_lca_w_EoL.lca.technosphere_matrix,
        row_labels=[f"{bw2data.get_activity(x)['name']} ({bw2data.get_activity(x)['location']})" for x in tech_lca_w_EoL.lca.product_dict.keys()],
        col_labels=[f"{bw2data.get_activity(x)['reference product']} [{bw2data.get_activity(x)['unit']}]" for x in tech_lca_w_EoL.lca.activity_dict.keys()]
    )
)

<IPython.core.display.Latex object>

In [53]:
bw2data.projects.set_current("ecoinvent-3.11-cutoff-bw25")

In [54]:
bw2data.databases

Databases dictionary with 19 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff
	ei_cutoff_3.11_remind_SSP2-PkBudg1000_2025_SPS1 2025-11-18
	ei_cutoff_3.11_remind_SSP2-PkBudg1000_2035_SPS1 2025-11-18
	ei_cutoff_3.11_remind_SSP2-PkBudg1000_2050_SPS1 2025-11-18
	h2_pem
	lci-pass_cars
	test carina 2
	test cdr 1
	test cdr 2
	test david 2
	test david 3
	test david 4
	test david 5
	test david 6
	test eff 1 abs
	test eff 1 relative
	test eff 2 relative
	test1

In [55]:
biosphere = bw2data.Database("ecoinvent-3.11-biosphere")
len(biosphere)

9795

In [56]:
len(biosphere)*195

1910025

In [57]:
len(biosphere)*3000

29385000

In [58]:
lca.biosphere_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (3, 6)>

In [59]:
ei = bw2data.Database("ecoinvent-3.11-cutoff")

In [67]:
ei_lca = bw2calc.LCA({ei.random(): 1})
ei_lca.lci()
ei_lca.biosphere_matrix.shape

(3238, 25412)

/opt/homebrew/Caskroom/miniforge/base/envs/edges/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.27e+13)
  warnings.warn(msg, UmfpackWarning)


In [68]:
type(ei_lca.biosphere_matrix)

scipy.sparse._csr.csr_matrix

In [69]:
def csr_size(M):
    return (M.data.nbytes + M.indices.nbytes + M.indptr.nbytes)

size_bytes = csr_size(ei_lca.biosphere_matrix)
print(size_bytes / 1024**2, "MB")


6.207283020019531 MB


In [70]:
B = ei_lca.biosphere_matrix

import sys

mem = (
    B.data.nbytes +
    B.indices.nbytes +
    B.indptr.nbytes
)

print(f"{mem/1024/1024:.2f} MB")


6.21 MB


In [71]:
ei_lca.biosphere_matrix.nnz

541321